## 검색어 생성

In [1]:
# Schema
from typing_extensions import Annotated
from pydantic import Field, BaseModel

class QueryResult(BaseModel):
    query: Annotated[
        list[str],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description="가장 적절한 검색어들의 리스트, 길이 최소 3개/최대 5개", 
        )
    ]

In [2]:
# 프롬프트
from langchain_core.prompts import PromptTemplate

query_template = '''
주어진 제목과 설명을 바탕으로, 가장 연관성이 높은 논문과 데이터셋을 검색하려 합니다.

가장 의미적으로 관련성이 높은 논문과 데이터셋을 검색할 수 있을 쿼리를 만들어 주세요.

[조건]
1. 쿼리는 2~3단어로 구성되어야 합니다.
2. 검색 방식은 쿼리와 정확히 일치하는 내용이 있는 논문이나 데이터를 반환하는 형식입니다.

[Input]
연구 주제: {subject}
연구 설명: {description}

[Output]
가장 적절한 3~5개의 쿼리를 JSON으로 출력해주세요.
'''

query_prompt = PromptTemplate.from_template(query_template)

In [3]:
from langchain_openai import ChatOpenAI
import json

with open("../data/input_data.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)

subject, description = input_data['dataset_title_etc_main'], input_data['dataset_expl_etc_main']

prompt = query_prompt.invoke(
    {
        'subject': subject, 
        'description': description,
    }
)

# sLLM
sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

structured_sllm = sllm.with_structured_output(QueryResult)
res = structured_sllm.invoke(prompt)

print(res.query)

['Architectural Urbanism', 'Melbourne Seoul', 'KTA projects', 'public architecture', 'civic architecture']
